In [ ]:
import pandas as pd
from config import REPOSITORIES_FILE, FILTERED_FILE, JUPYTER_FILE

In [ ]:
df = pd.read_excel(REPOSITORIES_FILE, keep_default_na=False)
print('Total repositories: ',len(df))

### Removing repositories with no languages

In [ ]:
nolanguages = df[df.languages==0]
df.drop(df[df['languages'] == 0].index, inplace = True)
print('Total repositories after filtering repositories with "no languages":',len(df))

### Removing repositories whose names contain the words "courses", "cours", "curso" or "cursos"

In [ ]:
courses = df.loc[df.name.str.contains('course',case=False)]
df.drop(df.loc[df.name.str.contains('course',case=False)].index, inplace=True )
print('Total repositories after filtering "courses":', len(df))

In [ ]:
cours = df.loc[df.name.str.contains('cours',case=False)]
df.drop(df.loc[df.name.str.contains('cours',case=False)].index, inplace=True )
print('Total repositories after filtering "cours":', len(df))

In [ ]:
curso = df.loc[df.name.str.contains('curso',case=False)]
df.drop(df.loc[df.name.str.contains('curso',case=False)].index, inplace=True )
print('Total repositories after filtering "curso":', len(df))

In [ ]:
cursos = df.loc[df.name.str.contains('cursos',case=False)]
df.drop(df.loc[df.name.str.contains('cursos',case=False)].index, inplace=True )
print('Total repositories after filtering "cursos":', len(df))

### Removing repositories with no commits or no contributors

In [ ]:
emptycommits = df.loc[df['commits'] == '']
df.drop(df[df['commits'] == ''].index, inplace = True)
print('Total repositories after filtering repositories with no commits:',len(df))

In [ ]:
nocontributors = df[df.contributors == 0]
df.drop(df[df['contributors'] == 0].index, inplace = True)
print('Total repositories after filtering repositories with no contributors:',len(df))

### Removing repositories where Primary Language is not in the Top 10 delimited by the Corpus

In [ ]:
### This seems wrong, fix later

#filters repositories by language, keeping just the ones that use the top 10 languages in the corpus
df2 = df.groupby('primaryLanguage').filter(lambda x: len(x) >= 18)
len(df2)

### Saving the remaining repositories to Filtered Repostiores

In [ ]:
#saves filtered projetcts to Excel
#removes timezone from dates, since Excel does not know how to handle that
df.createdAt = pd.to_datetime(df.createdAt).dt.tz_localize(None)
df.pushedAt = pd.to_datetime(df.pushedAt).dt.tz_localize(None)
df.to_excel(FILTERED_FILE, index=False)

In [ ]:
print(f"Total Disk Usage: {df['diskUsage'].sum()}")

In [ ]:
df2 = df.query("primaryLanguage== 'Jupyter Notebook'")
df2[:10].to_excel(JUPYTER_FILE, index=False)